In [ ]:
import requests  # pip install requests
from pprint import pprint
import json
from datetime import datetime
import time

For more details, read the [docs](https://developers.triathlon.org/docs/) and the [references](https://developers.triathlon.org/reference).

Get your API key for free on [developers.triathlon.org](https://developers.triathlon.org/) and save it in a `api_key.txt` file.

In [ ]:
with open("api_key.txt", "r") as f:
    api_key = f.readline()
headers = {'apikey': api_key}

In [ ]:
url_prefix = "https://api.triathlon.org/v1/"

In [ ]:
def get_request(url, params=""):
    print(url)
    response = requests.request("GET", url, headers=headers, params=params)
    d = json.loads(response.text)
    d = d["data"]
    return d

In [ ]:
# option1: get all participants of all 2019 WTS
url_suffix = "events?category_id=351&start_date=2019-01-01&end_date=2020-01-01&target_property=event.event_id&group_by=program.name"
res = get_request(url_prefix + url_suffix)
event_ids = [e["event_id"] for e in res]
event_titles = [e["event_title"] for e in res]

# option2: get all participants of the Olympic Qualification Event
# 130072, '2019 Tokyo ITU World Olympic Qualification Event'
event_ids = [130072]
event_titles = ['2019 Tokyo ITU World Olympic Qualification Event']

z = zip(event_ids, event_titles)
for i in z:
    print(i)

In [ ]:
def get_prog_id(event_id, prog_name = 'Elite Men'): 
    url = "{}events/{}/programs".format(url_prefix, str(event_id))
    res = get_request(url)
    # print(res)
    rr = next((r for r in res if r["prog_name"] == prog_name), None)
    if rr is not None:
        print("{} - {} - {}".format(rr["prog_date"], rr["event_id"], rr["prog_id"]))
        return rr["prog_id"]
        
event_prog_ids = []
for event_id in event_ids:
    prog_id = get_prog_id(event_id)
    if prog_id is not None:
        event_prog_ids.append([event_id, prog_id])
print(event_prog_ids)

In [ ]:
def get_athlete_ids(event_id, prog_id):
    url = "https://api.triathlon.org/v1/events/{}/programs/{}/entries".format(str(event_id), str(prog_id))
    res = get_request(url, params={"type":"start"})
    entries = res["entries"]
    # pprint(entries)
    athlete_ids = []
    athlete_slugs = []
    for a in entries:
        athlete_ids.append(a["athlete_id"])
        athlete_slugs.append(a["athlete_slug"])
    return athlete_ids, dict(zip(athlete_ids, athlete_slugs))

In [ ]:
for event_id, event_prog in event_prog_ids:
    athlete_ids, athletes_mappings = get_athlete_ids(event_id, event_prog)    

In [ ]:
pprint(athletes_mappings)

In [ ]:
entries_limit = 20
def get_stats_res_one_athlete(athlete_id):
    url = "https://api.triathlon.org/v1/athletes/{}/results".format(athlete_id)
    res = get_request(url, params={"type":"start", "per_page":str(entries_limit), "category_id":"351"})
    stats = []
    for r in res:
        categories = [c["cat_id"] for c in r["event_categories"]]
        if 351 in categories:
            stats.append([r["result_api_listing"], r["position"]])
        else:
            "not a WTS"  # should not happen
    return stats

In [ ]:
def str_to_time(str_):
    time_ = datetime.strptime(str_, '%H:%M:%S')
    return time_

def get_gap(result_api_listing, position):
    res = get_request(result_api_listing)
    try:
        previous_time = res["results"][position-1]["total_time"]
        current_time = res["results"][position]["total_time"]
        next_time = res["results"][position+1]["total_time"]

        previous_time = str_to_time(previous_time)
        current_time = str_to_time(current_time)
        next_time = str_to_time(next_time)

        gap_before = (current_time - previous_time).total_seconds()
        gap_after = (next_time - current_time).total_seconds()
        return [gap_before, gap_after]
    except Exception as e:
        print("exception: {}".format(e))
        return [None, None]

In [ ]:
start = time.time()
athlete_counter = 0
athletes_gaps = {}
for athlete_id in athlete_ids:  # [0:2]
    athlete_counter += 1
    stats = get_stats_res_one_athlete(athlete_id)
    print("{} results for {}:".format(len(stats), athletes_mappings[athlete_id]))
    gaps_before = []
    gaps_after = []
    for result_api_listing, position in stats:
        # todo: special case if first
        if isinstance(position, int) and position > 1:
            gap_before, gap_after = get_gap(result_api_listing, position)
            if gap_before is not None:
                gaps_before.append(gap_before)
            if gap_after is not None:
                gaps_after.append(gap_after)
    athletes_gaps[athlete_id] = [gaps_before, gaps_after]
    print("{}: {}".format(athletes_mappings[athlete_id], [gaps_before, gaps_after]))
    print("counter = {} -- intermediate duration = {:.2f} s".format(athlete_counter, time.time() - start))
print("total duration = {:.2f} s".format(time.time() - start))

In [ ]:
print("len(athletes_gaps) = {}".format(len(athletes_gaps)))

In [ ]:
read_from_file = False
write_to_file = True

In [ ]:
if write_to_file:
    to_write = {"athletes_gaps": athletes_gaps,
                "athletes_mappings": athletes_mappings}
    with open("athletes_gaps_and_mappings.json", "w") as f:
        json.dump(to_write, f)

In [ ]:
if read_from_file:
    with open("athletes_gaps_and_mappings.json") as f:
        loaded_data = json.load(f)
        athletes_mappings = loaded_data["athletes_mappings"]
        athletes_gaps = loaded_data["athletes_gaps"]

In [ ]:
for a_id, a_n in athletes_mappings.items():
    print(a_id)
    print(a_n)
    if a_id in athletes_gaps:
        print(athletes_gaps[a_id])

In [44]:
for a, gaps in athletes_gaps.items():
    print(athletes_mappings[a])
    gaps[0] = [x for x in gaps[0] if x is not None]
    if len(gaps[0]) > 0:
        print("   before: sub2-%={:.1f} mean={:.2f} min={} max={} len={}".format(100*sum(i < 2 for i in gaps[0])/len(gaps[0]), sum(gaps[0])/len(gaps[0]), min(gaps[0]), max(gaps[0]), len(gaps[0])))
    gaps[1] = [x for x in gaps[1] if x is not None]
    if len(gaps[1]) > 0:
        print("   after:  sub2-%={:.1f} mean={:.2f} min={} max={} len={}".format(100*sum(i < 2 for i in gaps[1])/len(gaps[1]), sum(gaps[1])/len(gaps[1]), min(gaps[1]), max(gaps[1]), len(gaps[1])))

roberto_sanchez_mantecon
   before: sub2-%=0.0 mean=5.00 min=2.0 max=10.0 len=3
   after:  sub2-%=0.0 mean=9.67 min=2.0 max=22.0 len=3
alessandro_fabian
   before: sub2-%=5.6 mean=9.44 min=1.0 max=50.0 len=18
   after:  sub2-%=22.2 mean=12.56 min=1.0 max=88.0 len=18
rostislav_pevtsov
   before: sub2-%=40.0 mean=3.53 min=0.0 max=19.0 len=15
   after:  sub2-%=26.7 mean=3.13 min=0.0 max=8.0 len=15
antonio_serrat_seoane
   before: sub2-%=15.8 mean=8.47 min=0.0 max=26.0 len=19
   after:  sub2-%=15.8 mean=5.16 min=0.0 max=20.0 len=19
lukas_hollaus
   before: sub2-%=26.7 mean=5.60 min=1.0 max=22.0 len=15
   after:  sub2-%=33.3 mean=8.47 min=0.0 max=46.0 len=15
leo_bergere
   before: sub2-%=11.8 mean=8.29 min=1.0 max=40.0 len=17
   after:  sub2-%=5.9 mean=5.47 min=1.0 max=20.0 len=17
dorian_coninx
   before: sub2-%=14.3 mean=6.57 min=1.0 max=16.0 len=14
   after:  sub2-%=21.4 mean=3.50 min=0.0 max=11.0 len=14
pierre_le_corre
   before: sub2-%=17.6 mean=6.00 min=0.0 max=36.0 len=17
   after:  s

In [ ]:
"""
url_suffix = "events/"
url_suffix = "statistics/results?analysis=count_unique&target_property=event.name"
url_suffix = "statistics/results?analysis=average&target_property=athlete.age&group_by=format|athlete.gender"
url_suffix = "statistics/results?analysis=count&filters=position,lte,10&group_by=athlete.country|athlete.gender&timeframe=this_year"
url_suffix = "statistics/results?analysis=minimum&target_property=position&filters=athlete.country,eq,JPN|athlete.gender,eq,male"
url_suffix = "statistics/results?analysis=minimum&target_property=position&filters=athlete.country,eq,JPN|athlete.gender,eq,male|position,eq,7&group_by=athlete.name|event.name"
url_suffix = "statistics/results?analysis=count&filters=athlete.name,eq,Alistair%20Brownlee|position,lte,3&group_by=position"
url_suffix = "statistics/results?analysis=count_unique&target_property=event.name&group_by=event.name|program.id|program.name"
"""